## Descripción contextual del problema
Una clínica médica debe seleccionar un subconjunto de pruebas diagnósticas (representadas por características del dataset load_breast_cancer de sklearn) para incluir en un paquete básico de exámenes preventivos. Cada prueba tiene:

* Un costo asociado.
* Un nivel de importancia clínica basado en su correlación con el diagnóstico.
* Un límite total de presupuesto para el paquete.

El objetivo es maximizar la utilidad total de las pruebas seleccionadas sin exceder el presupuesto disponible.

### Identificación del tipo de problema

* Determinista
* Lineal
* Con restricciones
* Variables Discretas (0 o 1 por prueba seleccionada)

### Formulación matemática

Variables de decisión:

* 𝑥𝑖 ∈ {0, 1}: 1 si seleccionamos la prueba 𝑖, 0 si no.

Función objetivo (maximizar):

$$
\text{Maximizar} \quad Z = \sum_{i=1}^{n} u_i x_i
$$

Donde:

* 𝑢𝑖: utilidad (ej., correlación con el diagnóstico) de la prueba 𝑖.

Restricción de presupuesto:

$$
\sum_{i=1}^{n} c_i x_i \leq B
$$

Donde:

* 𝑐𝑖: costo de la prueba 𝑖,
* 𝐵: presupuesto máximo disponible.

### Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Conjunto de datos real de cáncer de mama incluido en scikit-learn.
# Este dataset contiene 30 características (variables) que se usarán como base
from sklearn.datasets import load_breast_cancer

# La función linprog permite resolver problemas de programación lineal, es decir,
# encontrar el valor óptimo (máximo o mínimo) de una función lineal sujeta a restricciones también lineales.
from scipy.optimize import linprog

### Cargar dataset

In [ ]:
data = load_breast_cancer()
n_features = data.data.shape[1]

print(data.DESCR)

### Simulaciones

In [ ]:
# Simular costos de las pruebas (aleatorios entre 1 y 10)
np.random.seed(42)
costs = np.random.randint(1, 11, size=n_features)

# Simular utilidad como la correlación con el target
X = data.data
y = data.target

# Calcular qué tan relevante es cada atributo respecto al target, usando correlación de Pearson
# Se utiliza la transpuesta de X para seleccionar las correlaciones relevantes entre y y cada característica
utilities = np.abs(np.corrcoef(X.T, y)[-1, :-1])
# El resultado antes de 'np.abs' es un vector como: [0.65, -0.22, 0.41, ...]

# Se toma el valor absoluto de cada correlación porque la correlación negativa
# también significa relación fuerte, el signo no importa para medir relevancia

# Presupuesto máximo
budget = 60

# Convertir a problema de minimización (linprog minimiza por defecto)
c = -utilities  # porque queremos maximizar Z
A = [costs]
b = [budget]
bounds = [(0, 1) for _ in range(n_features)]  # variables binarias entre 0 y 1

# Resolver con linprog
res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

### Mostrar resultados

In [ ]:
print("Valor óptimo (utilidad total):", -res.fun)
print("Presupuesto usado:", np.dot(costs, res.x))

print("\nPruebas seleccionadas:")
costos = 0
for i, val in enumerate(res.x):
    if val == 1:  # considerar como seleccionada
        print(f"- {data.feature_names[i]} (costo: {costs[i]}, utilidad: {utilities[i]:.2f})")
        costos += costs[i]

print("\nCantidad de pruebas seleccionadas:", np.sum(res.x == 1.0))
print("Presupuesto usado realmente:", costos)

# La diferencia con el presupuesto inicial (60) se debe a que np.dot(costs, res.x)
# multiplica cada costo por el valor de la variable óptima. Como se muestra en la tabla siguiente,
# la última prueba tiene un costo de 4 y un valor óptimo de 0.25, lo que aporta 1 al total,
# pero no debe considerarse, ya que solo se cuentan las pruebas cuyo valor de variable es 1.

print("\nÍndice | Costo   | Variable óptima")
print("----------------------------------")
for i, (c, x) in enumerate(zip(costs, res.x)):
    print(f"{i:6d} | {c:7.2f} | {x:15.3f}")